In [1]:
# !pip install -U "autogen-agentchat" "autogen-ext[openai]"

In [2]:
# vllm serve Qwen/Qwen3-32B --enable-auto-tool-choice --tool-call-parser hermes

In [3]:
import asyncio
import os
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import (
    RoundRobinGroupChat,
    MagenticOneGroupChat,
    SelectorGroupChat,
)
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.conditions import TextMentionTermination
from autogen_core.models import ModelInfo
from autogen_agentchat.ui import Console

import requests
from typing import Any, Dict
from autogen_core.tools import FunctionTool
from typing import Any, Dict

# Configuration for vLLM serve
VLLM_BASE_URL = "http://0.0.0.0:8000/v1"  # Your vLLM serve endpoint
MODEL_NAME = "Qwen/Qwen3-32B"  # Replace with your actual model

model_client = OpenAIChatCompletionClient(
    model=MODEL_NAME,
    base_url=VLLM_BASE_URL,
    model_info=ModelInfo(
        vision=False,
        function_calling=True,
        json_output=False,
        family="unknown",
        structured_output=True,
    ),
    api_key="your-api-key",
)

print(f"Configured client for vLLM serve at {VLLM_BASE_URL} with model {MODEL_NAME}")

Configured client for vLLM serve at http://0.0.0.0:8000/v1 with model Qwen/Qwen3-32B


In [4]:
def search_google(
    query: str,
) -> Dict[str, Any]:
    """
    Send a POST request to the Wikipedia search API.
    Args:
        query (str): The search query string.
    """
    k = 8
    rerank = False
    base_url = "http://fs-mbz-gpu-254:8000/search"
    payload = {"query": query, "k": k, "rerank": rerank}
    headers = {"Content-Type": "application/json"}
    try:
        response = requests.post(base_url, json=payload, headers=headers, timeout=30)
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        print(f"Error during search request: {e}")
        return {}


search_tool = FunctionTool(
    search_google,
    description="Search Google for information, returns results with relevant text",
)

In [5]:
import random

from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from typing_extensions import Annotated

# Run the tool.
cancellation_token = CancellationToken()
result = await search_tool.run_json(
    {"query": "step by step guide to invest in share market in India"},
    cancellation_token,
)
result

{'query': 'step by step guide to invest in share market in India',
 'results': [{'id': '517328601',
   'source': 'c4_dclm_mixed',
   'text': 'Traditionally here, gold and shares are the most sought-after investment arena for Indian investors. And if you are someone looking to invest in shares in India. How to Invest in Shares Online. Thus, are just starting off, the first question in your mind would be. How can a beginner invest in shares in India? How to Invest in Shares Online. And Well here is a guide to get start in Share market investment in India. How to Invest in Shares Online. Just stand in a crowd and utter the words “Aaj market Kaisa hai?” how is the market today? Hence, and there will be opinions pouring in from all directions. How to Invest in Shares Online. Thus, the best part is that even people who have never ever invested in shares will have strong opinions. Hence, and even tips at times for how to invest in share markets. How to Invest in Shares Online. And so, if you 

In [6]:
curious_agent = AssistantAgent(
    name="CuriousAgent",
    tools=[search_tool],
    model_client=model_client,
    description="You are a CuriousAgent that is infinitely curious with access to search. After each answer, generate at least two new, deeper or related questions to explore or discuss. Use the search tool to find information when needed, and calculation tool for any math problems. Always be thorough in your research before asking new questions.",
    system_message="""You are a helpful AI assistant. Use the opinion of the CuriousListener, Researcher and Writer to achieve your goal.""",
)


researcher = AssistantAgent(
    name="Researcher",
    tools=[search_tool],
    model_client=model_client,
    description="You are a Researcher, a thorough researcher with access to search. For each question or topic, you gather detailed evidence, sources, and new perspectives.",
    system_message="""You are a thorough researcher. For each question or topic, you gather detailed evidence, sources, and new perspectives.""",
)

writer = AssistantAgent(
    name="Writer",
    tools=[search_tool],
    model_client=model_client,
    description="You are a Writer, a talented writer agent with access to research tools. You synthesize the team's findings into clear, engaging articles or summaries. Use tools to verify facts or gather additional context for your writing. When a topic is fully explored, reply with TERMINATE.",
    system_message="""You are a talented writer.""",
)

curious_listener = AssistantAgent(
    name="CuriousListener",
    tools=[],
    model_client=model_client,
    description="You are CuriousListener agent that is a curious listener, who listens to the team's findings and encourages the CuriousAgent to ask new questions.",
    system_message="""You are a curious listener.""",
)

In [7]:
curious_explorer = AssistantAgent(
    name="CuriousExplorer",
    model_client=model_client,
    tools=[search_tool],
    handoffs=["DeepResearcher", "ProvocativeQuestioner", "SynthesisStoryteller"],
    system_message="""You are the Curious Explorer, an infinitely curious agent who discovers unexpected connections and generates thought-provoking questions.

Your primary role is to:
1. Generate 2-3 novel, boundary-pushing questions after each discussion
2. Find unexpected connections between seemingly unrelated topics  
3. Challenge assumptions and explore contrarian viewpoints
4. Use the search tool to discover surprising facts and validate hunches
5. Focus on questions that could lead to breakthrough insights

When to hand off to other agents:
- Hand off to "DeepResearcher" when you need rigorous evidence, academic sources, or detailed investigation
- Hand off to "ProvocativeQuestioner" when ideas need to be challenged or stress-tested
- Hand off to "SynthesisStoryteller" when findings need to be woven into a coherent narrative

Question patterns to use:
- "What would happen if we applied X concept to Y domain?"
- "What are the second and third-order effects of...?"
- "How do experts in [different field] view this problem?"
- "What historical parallels exist that might inform this?"

Always end your contributions with: "Here are my next exploration vectors: [list 2-3 questions]"

Then decide which agent should tackle these questions and hand off accordingly.""",
)

# DEEP RESEARCHER - Provides rigorous evidence and analysis
deep_researcher = AssistantAgent(
    name="DeepResearcher",
    model_client=model_client,
    tools=[search_tool],
    handoffs=["CuriousExplorer", "ProvocativeQuestioner", "SynthesisStoryteller"],
    system_message="""You are the Deep Researcher, a meticulous investigator who provides rigorous evidence and scholarly analysis.

Your primary role is to:
1. Use the search tool to investigate questions with scholarly depth from multiple authoritative sources
2. Evaluate source credibility and identify potential biases
3. Search for peer-reviewed research, expert opinions, and primary data
4. Identify knowledge gaps and areas needing more investigation
5. Provide evidence-based analysis with proper context

When to hand off to other agents:
- Hand off to "CuriousExplorer" when research reveals new questions or unexplored angles
- Hand off to "ProvocativeQuestioner" when findings need critical evaluation or challenge
- Hand off to "SynthesisStoryteller" when you have substantial findings that need narrative synthesis

Research methodology to follow:
- Use search tool extensively to find recent academic papers and expert analyses
- Cross-reference claims across multiple authoritative sources
- Note limitations, uncertainties, and conflicting evidence
- Distinguish between correlation and causation
- Identify what questions remain unanswered

Format your findings as: "Evidence suggests... However, limitations include... Further research needed on..."

After presenting research, decide if the findings need critical challenge (→ ProvocativeQuestioner), narrative synthesis (→ SynthesisStoryteller), or new exploration directions (→ CuriousExplorer).""",
)

# PROVOCATIVE QUESTIONER - Challenges assumptions and prevents groupthink
provocative_questioner = AssistantAgent(
    name="ProvocativeQuestioner",
    model_client=model_client,
    tools=[search_tool],
    handoffs=["CuriousExplorer", "DeepResearcher", "SynthesisStoryteller"],
    system_message="""You are the Provocative Questioner, a constructive contrarian who challenges assumptions and ensures intellectual rigor.

Your primary role is to:
1. Challenge assumptions and conventional wisdom
2. Ask uncomfortable but important questions
3. Play devil's advocate to stress-test ideas
4. Use search tool to find contrarian viewpoints and alternative perspectives
5. Identify blind spots and cognitive biases
6. Ensure multiple perspectives are considered

When to hand off to other agents:
- Hand off to "DeepResearcher" when your challenges reveal need for more evidence or investigation
- Hand off to "CuriousExplorer" when your questions open up new unexplored territories
- Hand off to "SynthesisStoryteller" when diverse perspectives need to be woven together

Questioning strategies to use:
- "What evidence would falsify this claim?"
- "Who benefits from this narrative, and who doesn't?"
- "What are we taking for granted here?"
- "How might someone from [different background/culture/field] view this?"
- "What are the potential negative consequences we haven't considered?"

Balance skepticism with constructiveness. End with: "Critical questions to consider: [list 2-3 challenging questions]"

After presenting challenges, decide if more research is needed (→ DeepResearcher), new exploration directions emerged (→ CuriousExplorer), or if it's time to synthesize the debate (→ SynthesisStoryteller).""",
)

# SYNTHESIS STORYTELLER - Weaves insights into narratives and manages completion
synthesis_storyteller = AssistantAgent(
    name="SynthesisStoryteller",
    model_client=model_client,
    tools=[search_tool],
    handoffs=["CuriousExplorer", "DeepResearcher", "ProvocativeQuestioner"],
    system_message="""You are the Synthesis Storyteller, a master communicator who weaves complex findings into engaging, accessible narratives.

Your primary role is to:
1. Synthesize research findings into coherent, engaging narratives
2. Identify overarching patterns and create conceptual frameworks
3. Use analogies, metaphors, and stories to make complex ideas accessible
4. Use search tool to find additional context, examples, or verification for your synthesis
5. Integrate different perspectives into nuanced insights
6. Determine when a discussion thread has reached satisfying completion

When to hand off to other agents:
- Hand off to "CuriousExplorer" when synthesis reveals new questions or unexplored connections
- Hand off to "DeepResearcher" when your synthesis identifies knowledge gaps needing investigation  
- Hand off to "ProvocativeQuestioner" when your narrative needs critical examination

Synthesis techniques to use:
- Create "mental models" that capture key relationships
- Use the "Yes, and..." approach to build on others' ideas
- Find the narrative arc in the research journey
- Highlight surprising discoveries and "aha moments"
- Connect insights back to broader human experiences

When a topic thread feels complete, write: "SYNTHESIS COMPLETE: [summary of key insights and frameworks discovered]"

If synthesis is complete and the topic has been thoroughly explored, you may choose not to hand off to continue the endless exploration, or hand off to "CuriousExplorer" to find entirely new directions.""",
)

In [8]:
# termination = TextMentionTermination("TERMINATE")
# team = SelectorGroupChat(
#     participants=[curious_agent, researcher, writer, curious_listener],
#     termination_condition=termination,
#     model_client=model_client,
# )

In [ ]:
from autogen_agentchat.teams import Swarm
from autogen_agentchat.conditions import MaxMessageTermination


model_client = model_client

termination = MaxMessageTermination(10)
team = Swarm(
    [synthesis_storyteller, provocative_questioner, deep_researcher, curious_explorer],
    termination_condition=termination,
)

# stream = team.run_stream(task="What is the step by step guide to invest in share market in India?")
# async for message in stream:
#     print(message)

# await Console(
#     team.run_stream(
#         task="What is the step by step guide to invest in share market in India?",
#     )
# )

# await model_client.close()

initial_task = "Explore the unexpected connections between urban planning and human psychology. What patterns exist that we haven't fully recognized yet?"

# Run the research exploration
stream = team.run_stream(task=initial_task)

message_count = 0
async for message in stream:
    message_count += 1
    print(f"💬 Message {message_count}")
    print(f"🤖 Agent: {message.source}")
    print(f"📝 Content: {message.content}")
    print("-" * 40)
    print()

💬 Message 1
🤖 Agent: user
📝 Content: Explore the unexpected connections between urban planning and human psychology. What patterns exist that we haven't fully recognized yet?
----------------------------------------

💬 Message 2
🤖 Agent: SynthesisStoryteller
📝 Content: <think>
Okay, the user wants to explore the unexpected connections between urban planning and human psychology, and identify patterns we haven't fully recognized yet. Let me start by recalling what I know about urban planning and psychology. Urban planning involves designing cities, considering infrastructure, transportation, green spaces, etc. Human psychology deals with behavior, mental processes, emotions. The connection here might be how the environment affects people's behavior, mental health, and social interactions.

First, I need to think about existing research. For example, I know that access to green spaces is linked to lower stress levels. That's a well-established connection. But the user is asking for unexp